In [1]:
import torch
import torch.nn as nn
import torchvision

In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=in_channels,
                out_channels=
            )
        )